In [1]:
import tensorflow as tf
import numpy as np

In [2]:
seq_data = [['I','am','a','boy'], ['나','는','소년','이다'], ['I','am', 'a', 'girl'], ['나','는','소녀','다'],['You', 'are', 'so', 'beautiful'], ['당신','은','너무','아름답다']]

In [3]:
char_arr = []
for seq in seq_data:
    char_arr += seq
char_arr += ['<S>', '</S>', '<P>']
char2num = {c:i for i, c in enumerate(set(char_arr))}
num2char = {i:c for i, c in enumerate(char2num.keys())}
dic_len = len(char2num)

In [4]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    for i in range(len(seq_data)):
        input = []
        output = []
        target = []
        if i%2==0:
            for seq in seq_data[i]:
                input.append(char2num[seq])
            input_batch.append(np.eye(dic_len)[input])
        else:
            output.append(char2num['<S>'])
            for seq in seq_data[i]:
                output.append(char2num[seq])
                target.append(char2num[seq])
            target.append(char2num['</S>'])
            output_batch.append(np.eye(dic_len)[output])
            target_batch.append(target)

    return input_batch, output_batch, target_batch       

In [5]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 500
n_class = n_input = dic_len

In [6]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [7]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

In [9]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype=tf.float32)

In [10]:
logits = tf.layers.dense(outputs, n_class, activation=None)

In [12]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    if epoch % 50 == 49:
        print('Epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 050 cost = 0.149077
Epoch: 100 cost = 0.003479
Epoch: 150 cost = 0.002200
Epoch: 200 cost = 0.000995
Epoch: 250 cost = 0.001657
Epoch: 300 cost = 0.001592
Epoch: 350 cost = 0.001294
Epoch: 400 cost = 0.001153
Epoch: 450 cost = 0.000623
Epoch: 500 cost = 0.000150
optimization finished!


In [15]:
def translate(sentence):
    seq_data = [sentence.split(' '), len(sentence.split(' '))*['<P>']]
    input_batch, output_batch, target_batch = make_batch(seq_data)
    prediction = tf.argmax(logits, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    decoded = [num2char[i] for i in result[0]]
    
    end = decoded.index('</S>')
    translated = ' '.join(decoded[:end])
    
    return translated

In [16]:
print('===translation test===')
print('I am a boy ->', translate('I am a boy'))
print('I am a girl ->', translate('I am a girl'))
print('You are so beautiful ->', translate('You are so beautiful'))
print('I am so beautiful ->', translate('I am so beautiful'))

===translation test===
I am a boy -> 나 는 소년 이다
I am a girl -> 나 는 소녀 다
You are so beautiful -> 당신 은 너무 아름답다
I am so beautiful -> 나 는 소년 이다
